# Gallery Example: Hyper/Erl/1 Reentrant Queue

This example demonstrates a HyperExp/Erlang/1 queueing system with reentrant routing:
- **Arrivals**: Hyperexponential distribution (high variability)
- **Service**: Erlang distribution (low variability)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hyperl1_reentrant():
    """Create Hyper/Erl/1-Reentrant queueing model"""
    model = Network('Hyper/Erl/1-Reentrant')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # HyperExp arrival (SCV=64) for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, HyperExp.fit_mean_and_scv(1, 64))
    source.set_arrival(oclass2, Disabled())
    
    # Erlang service for Class1, Exponential for Class2
    queue.set_service(oclass1, Erlang.fit_mean_and_order(0.5, 5))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching (reentrant)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_class_switch(oclass1, oclass2, queue, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_hyperl1_reentrant()

## About This Model

This model combines:
- **High arrival variability** (HyperExp with SCV=64): Very bursty arrivals
- **Low service variability** (Erlang with SCV=0.2): Regular service times
- **Reentrant routing**: Jobs switch class and pass through again

The high SCV arrivals create significant queue buildup during bursts, while the regular Erlang service provides steady processing.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)